# 3.1

#### 3.1.1

In [74]:
import numpy as np
import igraph as ig
import matplotlib.pyplot as plt

def update_f(base, update_value):
    return 1/2*(base+update_value)
def aggregation_f(array):
    return np.mean(array)

def msg_passing(g,current_feature_rep,runs): #note that we need to change this if we want to input only a subgraph
    curr_r = current_feature_rep
    #assume that the initial feature representation spans the whole graph
    #get neighbors ONCE
        #neighborhood(vertices=None, order=1, mode='all', mindist=0): 
        #mindist = 1: exclude seed node
    neighbors=g.neighborhood(vertices=None,order=1,mode='all',mindist=1)
    #print(neighbors)
    results = [curr_r]
    for i in np.arange(runs):   
        next_r = np.empty_like(curr_r)
        for idx,n in enumerate(curr_r):
            #fetch the attributes out of the current representation array
            neigh_attr=[]
            for v in neighbors[idx]:
                neigh_attr.append(curr_r[v])     
            #aggregate the attributes of the neighbors
            aggregate = aggregation_f(neigh_attr)
            #update the current representation
            update = update_f(current_feature_rep[idx],aggregate)
            next_r[idx] = update
        curr_r = next_r
        results.append(next_r)
    return results

In [60]:
#test graph! (from sheet 2)
g = ig.Graph(
    n=5, edges=[(0,2),(0,3),(1,2),(2,3),(3,4),(1,4)],
    vertex_attrs={'num': [1, 2, 1, 1,3]}
)
#set the intial representation
init_feature_rep = np.array([1, 2, 1, 1, 3], dtype=float)

#run the message passing framework x = 2 times
result = msg_passing(g,init_feature_rep,2) #fix/set data types of arrays???
print(result)

[array([1., 2., 1., 1., 3.]), array([1.        , 2.        , 1.16666667, 1.33333333, 2.25      ]), array([1.125     , 1.85416667, 1.22222222, 1.23611111, 2.33333333])]


#### 3.1.2

In [61]:
g = ig.Graph.Read_Pickle('ogbn-arxiv.pickle').as_undirected()
init_feature_rep = np.array([v['attr'] for v in g.vs()])
print(init_feature_rep.shape)

(169343, 128)


In [62]:
runs=100
result = msg_passing(g,init_feature_rep,runs)
print(result)

[array([[-0.057943, -0.05253 , -0.072603, ...,  0.173364, -0.172796,
        -0.140059],
       [-0.1245  , -0.070665, -0.325202, ...,  0.068524, -0.372111,
        -0.301036],
       [-0.080242, -0.023328, -0.183787, ...,  0.109919,  0.117589,
        -0.139883],
       ...,
       [-0.22053 , -0.036568, -0.402199, ...,  0.11336 , -0.161393,
        -0.145171],
       [-0.138236,  0.040885, -0.251811, ..., -0.08929 , -0.041253,
        -0.376132],
       [-0.029875,  0.268417, -0.161124, ...,  0.120807,  0.077647,
        -0.091018]], dtype=float32), array([[-0.01890475, -0.01619825, -0.02623475, ...,  0.09674875,
        -0.07633124, -0.05996274],
       [-0.0577582 , -0.03084071, -0.1581092 , ...,  0.03875379,
        -0.1815637 , -0.14602621],
       [-0.0314514 , -0.0029944 , -0.0832239 , ...,  0.0636291 ,
         0.0674641 , -0.0612719 ],
       ...,
       [-0.09828539, -0.00630439, -0.18911989, ...,  0.0686596 ,
        -0.0687169 , -0.06060589],
       [-0.05939806,  0.030162

#### 3.1.3

In [91]:
result_array = np.asarray(result)

# get vertices by year
vert_year = np.array(g.vs['year']).squeeze()
# seperate vertices into two sets V_1, V_2
vert_ids_1 = np.argwhere(vert_year < 2019)
vert_ids_2 = np.argwhere(vert_year >= 2019)

# prepare the data:
# document representation as predictor (x), subject as response (y)

y_train = np.array(g.vs['label'])[vert_ids_1].squeeze()
y_test = np.array(g.vs['label'])[vert_ids_2].squeeze()

(48603,)


In [92]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0)

k_array = np.array([0,1])#,2,3,4,5,10,20,30,40,50,75,100])
score = np.empty_like(k_array)
for idx,k in enumerate(k_array):
    print(k)
    x_train = np.array((result_array[k]))[vert_ids_1].squeeze()
    x_test = np.array((result_array[k]))[vert_ids_2].squeeze()
    lr.fit(x_train,y_train)
    score[idx] = lr.score(x_test,y_test)
print(score)

0
(48603, 128)


C:\Users\mwibc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1
(48603, 128)


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(k_array,score)
plt.show()